Necessary Libraries

In [37]:
import requests
import pandas as pd
from pathlib import Path

API_KEY = "36e399c8efddfc967109cac3eaba15c58ccf99d9"

Retrieve Data

In [35]:
years = [y for y in range(2011, 2024) if y != 2020]
all_data = []

for year in years:
    url = f"https://api.census.gov/data/{year}/acs/acs1"
    params = {
        "get": "NAME,B12501_001E,B12501_005E,B12501_010E,B12503_005E,B12503_010E",
        "for": "state:*",
        "key": API_KEY
    }

    response = requests.get(url, params=params)

    try:
        data = response.json()
    except ValueError:
        print(f"{year}: JSON failed — skipped")
        continue

    df = pd.DataFrame(data[1:], columns=data[0])
    df["year"] = year
    #print(df)
    all_data.append(df)


Clean Data

In [36]:
#Declare empty dataframe
final_df = pd.DataFrame()

# Combine all years
final_df = pd.concat(all_data, ignore_index=True)

# Rename for clarity
final_df = final_df.rename(columns={
    "B12501_001E": "population_over_15",
    "B12501_005E": "married_males_last_year",
    "B12501_010E": "married_females_last_year",
    "B12503_005E": "divorced_males_last_year",
    "B12503_010E": "divorced_females_last_year"
})

cols = [
    "population_over_15",
    "married_males_last_year",
    "married_females_last_year",
    "divorced_males_last_year",
    "divorced_females_last_year"
]

for col in cols:
    final_df[col] = pd.to_numeric(final_df[col], errors="coerce")

# Compute totals
final_df["married_last_year"] = (
    final_df["married_males_last_year"] +
    final_df["married_females_last_year"]
)

final_df["divorced_last_year"] = (
    final_df["divorced_males_last_year"] +
    final_df["divorced_females_last_year"]
)
final_df.head()

,NAME,population_over_15,married_males_last_year,married_females_last_year,divorced_males_last_year,divorced_females_last_year,state,year,married_last_year,divorced_last_year
0,Alabama,3874758,37987,37530,22071.0,28381.0,01,2011,75517,50452.0
1,Alaska,565162,7766,7771,3398.0,4071.0,02,2011,15537,7469.0
2,Arizona,5129409,48065,46600,24130.0,30961.0,04,2011,94665,55091.0
3,Arkansas,2342376,25519,24025,14474.0,14026.0,05,2011,49544,28500.0
4,California,30062390,250693,238508,115216.0,124512.0,06,2011,489201,239728.0


Save to CSV

In [39]:
final_df.to_csv("../data/Marriage_Divorce_rates.csv")